In [1]:
# Downgrade pillow to avoid `UserWarning: Possibly corrupt EXIF data.`
!pip install pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 8.5MB/s eta 0:00:01
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [3]:
import os
# !git clone https://github.com/Tony607/efficientnet_keras_transfer_learning
# %cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
  !git clone https://github.com/Tony607/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/

/home/ec2-user/SageMaker/eye-Image-Analysis/efficientnet_keras_transfer_learning


In [4]:
!pip install keras

    100% |████████████████████████████████| 378kB 32.8MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

Using TensorFlow backend.


In [6]:
TARGET_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 100
IMAGE_SHAPE = (128, 128, 1)
IMAGE_SHAPE_EFFI = (128, 128, 3)


batch_size = 32

width = 256
height = 256
epochs = 100
# NUM_TRAIN = 2000
# NUM_TEST = 1000
# dropout_rate = 0.2
input_shape = (height, width, 3)

In [7]:
# loading pretrained conv base model
model = Net(weights='imagenet', include_top=False, input_shape=input_shape)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
16719872/16717576 [==============================] - 3s 0us/step


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 128, 128, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
swish (Swish)                   (None, 128, 128, 32) 0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
depthwise_

In [9]:
import os
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
sys.path.append('..')

In [10]:
%ls -la

total 220
drwxrwxr-x  7 ec2-user ec2-user   4096 Dec 13 13:04 ./
drwxrwxr-x 14 ec2-user ec2-user   4096 Dec 13 17:24 ../
drwxrwxr-x  3 ec2-user ec2-user   4096 Dec 13 14:01 efficientnet/
drwxrwxr-x  7 ec2-user ec2-user   4096 Dec 13 13:10 efficientnet_keras_transfer_learning/
drwxrwxr-x  2 ec2-user ec2-user   4096 Dec 13 13:03 examples/
drwxrwxr-x  8 ec2-user ec2-user   4096 Dec 13 13:03 .git/
-rw-rw-r--  1 ec2-user ec2-user   1274 Dec 13 13:03 .gitignore
-rw-rw-r--  1 ec2-user ec2-user     27 Dec 13 13:03 __init__.py
-rw-rw-r--  1 ec2-user ec2-user 173397 Dec 13 13:03 Keras_efficientnet_transfer_learning.ipynb
-rw-rw-r--  1 ec2-user ec2-user     17 Dec 13 13:03 MANIFEST.in
drwxrwxr-x  2 ec2-user ec2-user   4096 Dec 13 13:03 misc/
-rw-rw-r--  1 ec2-user ec2-user   3626 Dec 13 13:03 README.md
-rw-rw-r--  1 ec2-user ec2-user   3700 Dec 13 13:03 setup.py


In [11]:
%cd ..

/home/ec2-user/SageMaker/eye-Image-Analysis


In [12]:
%ls -la

total 74564
drwxrwxr-x 14 ec2-user ec2-user     4096 Dec 13 17:24 ./
drwxr-xr-x  5 ec2-user ec2-user     4096 Dec 13 17:30 ../
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data/
drwxrwxr-x  4 ec2-user ec2-user     4096 Dec 13 01:48 Database/
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data_equalized/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_partial_excluded/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_Suspect/
drwxrwxr-x  7 ec2-user ec2-user     4096 Dec 13 13:04 efficientnet_keras_transfer_learning/
-rw-rw-r--  1 ec2-user ec2-user     1487 Dec 13 01:48 evaluation_test.py
drwxrwxr-x  8 ec2-user ec2-user     4096 Dec 13 17:33 .git/
-rw-rw-r--  1 ec2-user ec2-user     1212 Dec 13 01:48 .gitignore
-rw-rw-r--  1 ec2-user ec2-user     4859 Dec 13 01:48 Graphs.py
-rw-rw-r--  1 ec2-user ec2-user    15614 Dec 13 17:13 Interpreter.py
-rw-rw-r--  1 ec2-user ec2-user    11233 Dec 13 02:25 Interpreter.pyc
drwxrwxr-x  2 ec2-user ec2-user     4096 

In [13]:
# !pip install keras
# !pip install git+https://github.com/qubvel/segmentation_models
from Interpreter import Interpreter

In [14]:
TARGET_SIZE = (256, 256)
BATCH_SIZE = 16
EPOCHS = 100
IMAGE_SHAPE = (256, 256, 1)
IMAGE_SHAPE_EFFI = (256, 256, 3)

inter = Interpreter(
    BATCH_SIZE,
    IMAGE_SHAPE_EFFI,
    EPOCHS,
    TARGET_SIZE
)

In [15]:
train_images, validation_images, test_images = inter.split_data()

Found 706 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [16]:
%cd efficientnet_keras_transfer_learning/

/home/ec2-user/SageMaker/eye-Image-Analysis/efficientnet_keras_transfer_learning


In [17]:


# loading pretrained conv base model
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)



In [18]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))
model.add(layers.Dropout(0.2, name="dropout_out"))
# model.add(layers.Dense(256, activation='relu', name="fc1"))
model.add(layers.Dense(2, activation='softmax', name="fc_out"))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 8, 8, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 4,010,110
Non-trainable params: 42,016
_________________________________________________________________


In [20]:
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable layers before freezing the conv base: 213
This is the number of trainable layers after freezing the conv base: 2


In [21]:
% cd ..
% ls

/home/ec2-user/SageMaker/eye-Image-Analysis
Data/                                  model_simple_128.json
Database/                              model_simple_64.h5
Data_equalized/                        model_simple_64.json
Data_partial_excluded/                 model_simple.h5
Data_Suspect/                          model_simple.json
efficientnet_keras_transfer_learning/  pre_proc.py
evaluation_test.py                     Processing/
Graphs.py                              Processor.py
Interpreter.py                         __pycache__/
Interpreter.pyc                        README.md
model1_accuracy_128.png                references/
model1_accuracy_64.png                 requirements.txt
model1_accuracy.png                    test_interpreter.py
model1_loss_128.png                    test_processor.py
model1_loss_64.png                     train_model.ipynb
model1_loss.png                        train_start.py
model.h5                               Untitled.ipynb
model_simple_128.h5   

In [ ]:
model.compile(
#     loss='categorical_crossentropy',
    loss='sparse_categorical_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc']
)
history = model.fit_generator(
      train_images,
      steps_per_epoch= len(train_images.classes) //batch_size,
      epochs=epochs,
      validation_data=validation_images,
      validation_steps= len(validation_images.classes) //batch_size,
#       verbose=1,
#       use_multiprocessing=True,
      workers=2
)

Epoch 1/100
45/45 [==============================] - 364s 8s/step - loss: 2.1416 - acc: 0.5142 - val_loss: 1.3823 - val_acc: 0.5571
Epoch 2/100
45/45 [==============================] - 463s 10s/step - loss: 2.3016 - acc: 0.5085 - val_loss: 1.6760 - val_acc: 0.5429
Epoch 3/100
45/45 [==============================] - 366s 8s/step - loss: 2.0149 - acc: 0.5297 - val_loss: 1.7204 - val_acc: 0.5571
Epoch 4/100
45/45 [==============================] - 449s 10s/step - loss: 2.2108 - acc: 0.5496 - val_loss: 1.8277 - val_acc: 0.4857
Epoch 5/100
45/45 [==============================] - 431s 10s/step - loss: 2.1644 - acc: 0.5071 - val_loss: 1.7732 - val_acc: 0.5286
Epoch 6/100
45/45 [==============================] - 341s 8s/step - loss: 2.3073 - acc: 0.5028 - val_loss: 1.8543 - val_acc: 0.5071
Epoch 7/100
45/45 [==============================] - 467s 10s/step - loss: 2.0802 - acc: 0.5042 - val_loss: 1.7203 - val_acc: 0.5357
Epoch 8/100
45/45 [==============================] - 400s 9s/step - loss